# Retrieval(탐색)

(단순히 문서를 쪼갠 상태에서 탐색을 수행하게 되면 애로사항이 생길 수 있습니다.  
query와 유사도가 높은 chunk를 반환하는 것이 보장되지 않기 때문이죠.  
이를 해결하기 위해 관련성이 높은 split들에 대해 index를 부여하고 이를 기준으로 필요한 문서를 가져오게 됩니다.)

탐색은 retrieval augmented generation (RAG) flow에 있어서 가장 중요한 요소입니다.

이전의 vectorDB를 불러와보죠.

## Vectorstore 탐색

In [ ]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [ ]:
#!pip install lark

### 유사도 검색

chroma를 통해 vectorstore를 세팅하는 과정은 이전 강의와 동일합니다.

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'

In [ ]:
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

209개로 나뉩니다.

In [ ]:
print(vectordb._collection.count())

In [ ]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [ ]:
smalldb = Chroma.from_texts(texts, embedding=embedding)

In [ ]:
question = "Tell me about all-white mushrooms with large fruiting bodies"

In [ ]:
smalldb.similarity_search(question, k=2)

In [ ]:
smalldb.max_marginal_relevance_search(question,k=2, fetch_k=3)

### 다양성 다루기: Maximum marginal relevance

지난 강의에서 한 문제를 다뤘습니다: 어떻게 하면 검색 결과에 다양성을 불어넣을 수 있을까?
 
`Maximum marginal relevance` query에 대한 연관성과 결과들의 *다양성*, 두 마리 토끼를 모두 잡는 방식입니다.

In [ ]:
question = "what did they say about matlab?"
docs_ss = vectordb.similarity_search(question,k=3)

단순히 유사도 검색을 하면 둘 다 동일한 결과가 출력됩니다.

In [ ]:
docs_ss[0].page_content[:100]

In [ ]:
docs_ss[1].page_content[:100]

`MMR` 방식을 적용했을 때의 차이에 주목해보세요

In [ ]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)

In [ ]:
docs_mmr[0].page_content[:100]

In [ ]:
docs_mmr[1].page_content[:100]

### 특정성 다루기: 메타데이터

지난 강의에서 세 번째 강의에 대한 질문에 대한 답변에 다른 강의도 포함되는 것을 확인했습니다.

이 문제를 해결하기 위해 vectorstores는 `metadata` 관련 기능을 제공합니다.

`metadata`는 각 embedded 덩어리(chunk)에 대해 문맥(context)을 제공합니다.

In [ ]:
question = "what did they say about regression in the third lecture?"

In [ ]:
docs = vectordb.similarity_search(
    question,
    k=3,
    filter={"source":"docs/cs229_lectures/MachineLearning-Lecture03.pdf"}
)

In [ ]:
for d in docs:
    print(d.metadata)

### 특정성 다루기: self-query 탐색기를 활용하여 metadata를 다뤄보기

query 자체로부터 메타데이터를 추론하기!

이를 위해서 우리는 `SelfQueryRetriever`를 사용할 것인데, 이는 다음 내용들을 추출하기 위해 LLM을 사용합니다.

1. 벡터 탐색을 위한 `query` 문자열
2. 메타데이터 필터

대부분의 벡터 데이터베이스는 메타데이터 필터를 지원합니다. 따라서 이는 새로운 데이터베이스나 인덱스를 필요로 하지 않습니다.

In [ ]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [ ]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `docs/cs229_lectures/MachineLearning-Lecture01.pdf`, `docs/cs229_lectures/MachineLearning-Lecture02.pdf`, or `docs/cs229_lectures/MachineLearning-Lecture03.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [ ]:
document_content_description = "Lecture notes"
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

In [ ]:
question = "what did they say about regression in the third lecture?"

다음 줄을 처음 실행하게 되면 'predict_and_parse'가 손상되었다는 **경고 메세지**가 나타날 수 있습니다. 이는 무시해도 괜찮습니다.

In [ ]:
docs = retriever.get_relevant_documents(question)

In [ ]:
for d in docs:
    print(d.metadata)

### 추가적인 트릭: 압축(compression)

탐색된 문서의 품질을 개선하기 위한 또다른 접근 방식은 압축입니다.

query와 가장 관련성이 높은 정보는 아마 관계 없는 수많은 데이터에 묻혀있을 것입니다.

그렇다고 해서 당신의 어플리케이션을 통해 문서 전체를 전달하는 것은 훨씬 큰 LLM 호출 비용과 열등한 답변으로 이어질 가능성이 높습니다.

문맥적 압축은 이를 고치는 것을 의미합니다.

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [ ]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [ ]:
# Wrap our vectorstore
llm = OpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)

In [ ]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [ ]:
question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

## 다양한 테크닉을 결합하기

search_type을 mmr로 지정하는 것만으로도 문제를 해결할 수 있습니다.

In [ ]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type = "mmr")
)

In [ ]:
question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

## 다른 종류의 탐색

vectordb가 문서 탐색의 유일한 방식이 아니라는 것을 상기할 필요가 있습니다.

`LangChain` 추상 탐색은 TF-IDF 또는 SVM과 같은 다른 문서 탐색 방식을 포함합니다.

In [ ]:
from langchain.retrievers import SVMRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# Load PDF
loader = PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture01.pdf")
pages = loader.load()
all_page_text=[p.page_content for p in pages]
joined_page_text=" ".join(all_page_text)

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits = text_splitter.split_text(joined_page_text)

어떤 탐색 결과가 나오는지 직접 실행하고 비교해 보세요

In [ ]:
# Retrieve
svm_retriever = SVMRetriever.from_texts(splits,embedding)
tfidf_retriever = TFIDFRetriever.from_texts(splits)

In [ ]:
question = "What are major topics for this class?"
docs_svm=svm_retriever.get_relevant_documents(question)
docs_svm[0]

In [ ]:
question = "what did they say about matlab?"
docs_tfidf=tfidf_retriever.get_relevant_documents(question)
docs_tfidf[0]